# Candy Crush in Python - Part 2 - Swapping

In this part of the tutorial we will be adding the first piece of functionality for our game - Swapping two neighbouring tiles.

## Code Recap

So far we have a game board representation using numpy arrays and a function for helping us display our board on the screen using the gamegrid libary:

In [1]:
import numpy as np
from gamegrid import GameGrid
from IPython.display import display

def show_board(board_array):
    grid = GameGrid()
    grid.data = board_array
    display(grid)

So we can display a board as follows:

In [2]:
board = np.array([
    [0, 1, 2],
    [3, 4, 0],
    [1, 2, 3]
])

show_board(board)

GameGrid()

## Swapping Values

The first function we will create is the swap function. This function will take our game board, two points and swap the values at those two points.

### Accessing Values in our board

If we want to swap the values in cells on our board, we need to know how to read the value at a given row and column, and how to change a value at a given row or column.

We can read our board in several ways, the first way is to get a whole row, we do this by accessing the board by 'index'. This means writing the name of our board variable, followed by the row number in square brackets:

In [3]:
row_0 = board[0]
print(row_0)

[0 1 2]


<div class="alert alert-warning">
    <b>Zero Indexing</b>
    <p>You will notice that in the example above, to get the first row we use the index 0. Data structures in python generally start with 0, so remember to get the first item you use 0, the second use 1, the 10th use 9.</p>
</div>

If we want a single cell from our row we can can access the row by index as well:

In [4]:
row_0 = board[0]
row_0_column_0 = row_0[0]
print(row_0_column_0)

0


We can also do this in a single step:

In [5]:
row_0_column_0 = board[0][0]
print(row_0_column_0)

0


We can do this by providing two indexes:

In [6]:
row_0_column_0 = board[0,0]
print(row_0_column_0)

0


For our game however we want to use python tuples. Tuples will let us represent a row and column together as a single thing.

#### Tuples

We represent a point (i.e. a row and column together) as a tuple in python. Creating a tuple is similar to a list, instead of using square brackets, we use round brackets, e.g. (1, 2) is a tuple with two values, 1 and 2. 

We represent a point on our game board as a tuple (row, column), so the tuple (0, 0) would be the first row, and the first column. The point (2, 1) would be the third row, second column (remember indexes start at 0):

In [7]:
point1 = (0,0)
print(point1)

point2 = (2,1)
print(point2)

(0, 0)
(2, 1)


We can access our the value at our points as follows:

In [8]:
row_0_column_0 = board[point1]
print(row_0_column_0)

row_2_column_1 = board[point2]
print(row_2_column_1)

0
2


### Setting a Value

To set a value on the board, we can simply do the following:

In [9]:
board[point1] = 2
print(board)

[[2 1 2]
 [3 4 0]
 [1 2 3]]


### Swap Function

So for our swap function, we will our board, and the two points we want to swap, called first and second, and do the following:
 
* Get the value at the first point and save it into a temporary variable called tmp
* Set the value at the first point to the value of the second point
* Set the value at the second point to the tempoarary variable we saved earlier

In [10]:
def swap_items(board, first, second):    
    # Save the value of the first cell in a temporary variable
    tmp = board[first]
    # Set the value of the first cell to the value of the second
    board[first] = board[second]        
    # Set the value of the second cell to the temporary variable we saved earlier
    board[second] = tmp   

We can test this function now, so lets create a board and display it:

In [11]:
swap_board = np.array([
    [0, 1, 2],
    [3, 4, 0],
    [1, 2, 3]    
])
show_board(swap_board)

GameGrid()

Now lets swap the values in cell (0,0) with the value in cell (0,1), and display our grid again.

In [12]:
# Swap (0,0) with (0, 1)
swap_items(swap_board, (0,0), (0,1))
# Display the game board
show_board(swap_board)

GameGrid()

In [13]:
# Swap (0,0) with (2, 2)
swap_items(swap_board, (0,0), (2,2))
# Display the game board
show_board(swap_board)

GameGrid()

### Swapping only Cells Beside eachother

We can swap any two cells, using this function, however in our game, we only want to allow swapping cells beside eachother. So lets add a check to make sure the cells are beside eachother. 

We must first define what does it mean for a cell to beside another cell? We can say that cells are beside eachother if one cell is to the left, right, directly above, or directly below. 

One way to do this is to subtract the first point from the second. This will give us an array with two values, the first is the 'distance' in rows, the second is the distance in columns.

In [18]:
first = (0,0)
second = (0,1)
diff = np.subtract(second, first)
print(diff)

[0 1]


So in the above example, the distance in rows is 0, and in columns is 1 - i.e. the second cell is directly below the first.

We now need to decide exactly what combinations of these two numbers are ok to swap - so lets look at some examples:

| Case | First | Second | Second - First | Beside? |
| ---- | ----- | ------ | -------------- | ------- |
| Left | (0, 0)| (0, 1) | (0, 1) | Yes |
| Below |  (0, 0)| (1, 0) | (1, 0) | Yes |
| Down and Left | (0, 0) | (1, 1) | (1, 1) | No |
| Right |(0, 1)| (0, 0) | (0, -1) | Yes |
| Above | (1, 0) | (0, 0) | (-1, 0) | Yes |
| Left 2 Cells | (0, 0) | (0, 2) | (0, 2) | No |
| Down 2 Cells | (0, 0) | (0, 2) | (2, 0) | No |

From this table we can see that a difference of (0, 1), (1, 0), (0, -1) and (-1, 0) are all ok to swap - and everything else is not. 

We could just check to make sure that the difference we get when we subtract is one of these four points - but if we look closely at each of the four examples, we can see that each of them has either a 1 or a -1, in it, and the other part of the point is zero. 

We can turn those -1's into 1's by using the np.abs() function, which gets the absolute value (i.e. the value without any minus sign) of each of the numbers in the point.

In [19]:
right_one = np.abs((0,-1))
print(right_one)
down_one = np.abs((-1, 0))
print(down_one)

[0 1]
[1 0]


Now we have only got to deal with two cases, (0, 1) and (1, 0), if we add the numbers in both of these points, we get 1 in both cases - and we do not get 1 for any of the cases where we do not want to swap. So we can use np.sum(..) to add all the values in a point together - so our check now becomes:

In [21]:
first = (0,0)
second = (0,1)
diff = np.subtract(second, first)
abs_value = np.abs(diff)
sum_values = np.sum(abs_value)

print(sum_values)

1


And our function becomes:

In [22]:
def swap_items(board, first, second):    
    # Calculate the 'distance'
    dist = np.sum(np.abs(np.subtract(second, first)))
    # Two cells are beside eachother if the distance is 1 or 0
    is_beside = dist <= 1
    # Only swap if they are beside
    if(is_beside):
        tmp = board[first]
        board[first] = board[second]        
        board[second] = tmp 

We can test this with a good swap:

In [23]:
print("Before Swap:")
show_board(swap_board)
# Swap (0,0) with (0, 1)
swap_items(swap_board, (0,0), (0,1))
# Display the game board
print("After Swap:")
show_board(swap_board)

Before Swap:


GameGrid()

After Swap:


GameGrid()

... and a bad one:

In [24]:
print("Before Swap:")
show_board(swap_board)
# Swap (0,0) with (2, 2)
swap_items(swap_board, (0,0), (2,2))
# Display the game board
print("After Swap:")
show_board(swap_board)

Before Swap:


GameGrid()

After Swap:


GameGrid()

## Summary

We are now able to swap two cells that are beside eachother. In the next lesson we will look at: [Matching Tiles](Part3-Matching.ipynb)

## Further Reading

* [Tuple Python Documentation](https://docs.python.org/3/tutorial/datastructures.html#tuples-and-sequences)
* [Tuples in Python Tutorial](https://www.tutorialspoint.com/python/python_tuples.htm)
* [Differences between Tuples and Lists](https://stackoverflow.com/questions/626759/whats-the-difference-between-lists-and-tuples)